In [1]:
import csv

In [ ]:
l0

Schedule saved to final_schedule.csv
Day summary saved to day_summary.csv

--- FINAL REPORT ---
Budget Given: ₹600.00
Budget Used: ₹599.64
Total Energy Consumption: 155.44 kWh

Number of 'ON' Hours per Appliance:
  Refrigerator: 720 hours (avg: 24.0 hours/day)
  Air Conditioner: 20 hours (avg: 0.7 hours/day)
  Lights: 142 hours (avg: 4.7 hours/day)
  Fans: 131 hours (avg: 4.4 hours/day)
  Television: 67 hours (avg: 2.2 hours/day)

Refrigerator Usage:
  Total Hours: 720
  Days with Usage: 30
  Maximum Hours per Day: 24
  Average Hours per Day: 24.0

Air Conditioner Usage:
  Total Hours: 20
  Days with Usage: 7
  Maximum Hours per Day: 4
  Average Hours per Day: 2.9
  Temperature Range: 31.8°C - 32.3°C
  Average Temperature: 32.1°C
  Maximum Continuous Hours: 4

Lights Usage:
  Total Hours: 142
  Days with Usage: 30
  Maximum Hours per Day: 6
  Average Hours per Day: 4.7

Fans Usage:
  Total Hours: 131
  Days with Usage: 22
  Maximum Hours per Day: 7
  Average Hours per Day: 6.0
  Temper